***This notebook contains a script that sends Student Contracts to Students for signing***

**Add local library to path**

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

***Import Libraries***

In [2]:
import pandas as pd
import re
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import send_bulk_signrequest

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


***Import Google WorkSheets***

In [3]:
#import student data
cohort_df = import_worksheet("Cohort 2022_ Data Review", "Cohort 2022_ DATA REVIEW")

***Rename Column Names***

In [4]:
#set new column names
column_names = {'First Name':'learner_name', 'Last Name':'learner_surname', 'Email Address':'learner_email',
                'Mobile Phone':'learner_phone', 'Date Of Birth':'learner_birthdate', 'Gender':'learner_gender',
                'ID / Passport Number':'learner_id', 'Middle Name(s)':'middle_name'}

#update target df with new column names
cohort_df = cohort_df.rename(columns=column_names)

***Convert Phone Numbers to String***

In [5]:
cohort_df['learner_id'] = cohort_df['learner_id'].astype(str)
cohort_df['learner_phone'] = [phone[-1] + phone[:-2] for phone in cohort_df['learner_phone'].astype(str)]

***Create Learner Title Column***

In [6]:
cohort_df["learner_title"] = ["Mr." if gender == "Male" else "Ms." for gender in cohort_df["learner_gender"]]

**Filter Active Students**

In [7]:
#cohort_df = cohort_df[cohort_df['Accepted provisional offer'] == 'Yes']
cohort_df = cohort_df[cohort_df['learner_surname'].notna()]


***Create Learner Full Name Column***

In [8]:
cohort_df['middle_name'] = cohort_df['middle_name'].fillna("")
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['learner_surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()

***Create Learner Initials Column***

In [9]:
cohort_df["learner_initials"] = [f"{name.split()[0][0]}.{name.split()[1][0]}" for name in cohort_df["learner_fullname"]]

**Add Template IDs**

In [10]:
cohort_df['student_contract'] = '435beaf0-66ae-41ad-961b-2506c9d99086'
cohort_df['code_of_conduct'] = '225f7ad2-539f-43b8-9350-f3f222b74e6c'

***Subset Relevant Columns***

In [11]:
#list relevant columns
contract_df_columns = ['learner_name', 'learner_surname', 'learner_title', 'learner_fullname', 'learner_initials',
                       'learner_email', 'learner_phone', 'learner_birthdate', 'learner_gender', 'learner_id',
                       'student_contract', 'code_of_conduct']

#subset columns based on list of relevant columns
cohort_df = cohort_df[contract_df_columns]

***Create Target Columns***

In [12]:
#student contract data
student_contract_columns = ['learner_name', 'learner_surname', 'learner_title', 'learner_fullname', 'learner_initials',
                       'learner_email', 'learner_phone', 'learner_birthdate', 'learner_gender', 'learner_id']

#code of conduct data
code_of_conduct_columns = ['learner_fullname', 'learner_initials']

**Subset particular students**

In [13]:
cohort_df = cohort_df[cohort_df['learner_email'].isin(['mentsha022@student.wethinkcode.co.za'])]


In [14]:
len(cohort_df)

1

***Create Test Dataframe***

In [283]:
#test_df = cohort_df.groupby(['student_contract']).head(1).groupby('student_contract').head(1).copy()
#test_df['learner_email'] = 'mufaro@thoughtquest.co.za'

**Subset Dataset**

In [75]:
#contract_df = contract_df[contract_df["Status after sign-request"] == "Resend Code"]
#contract_df = pd.merge(contract_df, missing_df, left_on="Email Address", right_on="Emails")
#contract_df.shape[0]

***Send Out SignRequests***

In [16]:
#send student contract signrequests in bulk
send_bulk_signrequest(cohort_df, student_contract_columns, 'student_contract')

#send code of conduct signrequests in bulk
send_bulk_signrequest(cohort_df, code_of_conduct_columns, 'code_of_conduct')

Signer: mentsha022@student.wethinkcode.co.za , Status: 201
Signer: mentsha022@student.wethinkcode.co.za , Status: 201


In [208]:
len(cohort_df)

1

In [290]:
short_df = cohort_df[cohort_df['learner_birthdate'].isnull()]

In [298]:
cohort_df.loc[cohort_df['learner_surname'] == 'Jockia', 'learner_birthdate'] = '20/10/1995'
cohort_df.loc[cohort_df['learner_surname'] == 'Mavhunga', 'learner_birthdate'] ='29/01/1991'

In [15]:
cohort_df.describe().transpose()

,count,unique,top,freq
learner_name,1,1,Melusi Lawrence,1
learner_surname,1,1,Ntshangase,1
learner_title,1,1,Mr.,1
learner_fullname,1,1,Melusi Lawrence Ntshangase,1
learner_initials,1,1,M.L,1
learner_email,1,1,mentsha022@student.wethinkcode.co.za,1
learner_phone,1,1,027658921748,1
learner_birthdate,1,1,08/12/1999,1
learner_gender,1,1,Male,1
learner_id,1,1,9912085193081,1
